### Using RGDR
This notebook outlines the current status of the RGDR implementation.

First we will load in some example data, and resample them using the `AdventCalendar`

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import s2spy.time
import s2spy.rgdr
from s2spy import RGDR

file_path = '../tests/test_rgdr/test_data'
field = xr.open_dataset(f'{file_path}/sst_daily_1979-2018_5deg_Pacific_175_240E_25_50N.nc')
target = xr.open_dataset(f'{file_path}/tf5_nc5_dendo_80d77.nc')

cal = s2spy.time.AdventCalendar((8, 31), freq = "30d")
cal = cal.map_to_data(field)
field_resampled = s2spy.time.resample(cal, field)
target_resampled = s2spy.time.resample(cal, target)

### The target timeseries comes from already pre-clustered land surface temperature data.

In [2]:
target_timeseries = target_resampled.sel(cluster=3).ts.isel(i_interval=0)
precursor_field = field_resampled.sst.isel(i_interval=1)

rgdr = RGDR(eps_km=600, alpha=0.05, min_area_km2=3000**2)

TypeError: RGDR.__init__() missing 1 required positional argument: 'timeseries'

Using `.plot_correlation` we can see the correlation and p-value map of the target timeseries and precursor field:

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 2))
_ = rgdr.plot_correlation(precursor_field, target_timeseries, ax1, ax2)

We can compare two RGDR initializations to preview what the effect on the generated clusters will be.

Note that the second figure has an extra cluster, due to the minimum area argument having a smaller value.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 2))

_ = rgdr.plot_clusters(precursor_field, target_timeseries, ax=ax1)

_ = RGDR(target_timeseries,
         eps_km=600,
         alpha=0.05,
         min_area_km2=1000**2
         ).plot_clusters(precursor_field, ax=ax2)

With `.fit` the RGDR clustering can be fit to a precursor field.
`.transform` can then be used to return the data reduced to clusters:

In [ ]:
rgdr.fit(precursor_field, target_timeseries)
clustered_data = rgdr.transform(precursor_field)
xr.set_options(display_expand_data=False) # Hide the full data repr
clustered_data

Now we can plot the data for each cluster:

In [ ]:
import matplotlib.pyplot as plt

clustered_data -= clustered_data.mean(dim='anchor_year')
clustered_data.sel(cluster_labels=1).plot.line(x='anchor_year', label='pos. corr')
clustered_data.sel(cluster_labels=-2).plot.line(x='anchor_year', label='neg. corr')
plt.legend()
